In [36]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from oracles import LogReg
from methods import Standard_Newton,\
Basic_method, PositiveCase_method, GeneralCase_method
from utils import function_plot_builder, bits_plot_builder, bits_plot_builder1
from methods import CubicMaxNewton, CubicMaxNewtonP
from methods import DINGO, GeneralCase_methodP, PositiveCase_methodP
from easydict import EasyDict
from methods import diana, adiana, dcgd
from utils import loss_logistic, grad

In [37]:
np.random.seed(42)

In [38]:
dataset_path = './Datasets/a2a.txt'
data_name = 'a2a'

In [39]:
# regularization parameter
lmb = 1e-3

In [40]:
# number of nodes, size of local data, and number of weights
N = 2265    
n = 15         
m = 151             
d = 123 

In [41]:
# data reading
b = np.zeros((N,))   
A = np.zeros((N, d))

f = open(dataset_path, 'r')
for i, line in enumerate(f):
    if i < N:
        line = line.split()
        for c in line:
            if c == '+1':
                b[i] = 1
            elif c == '-1':
                b[i] = -1
            elif c == '\n':
                continue
            else:
                c = c.split(':')
                A[i][int(c[0]) - 1] = float(c[1]) 
                
f.close()

In [42]:
# create logistic regression problem
logreg = LogReg(A=A, b=b, reg_coef=lmb, n=n, m=m, d=d)

In [ ]:
# find optimal solution using Newton's method
SN = Standard_Newton(logreg)
SN.find_optimum(np.zeros(d), n_steps=20)
x_opt = logreg.get_optimum()

In [44]:
# define shift
shift = np.ones(d)*0.1

## Newton's method

In [45]:
# run Newton's method
fv_nm, it_nm, bi_nm = SN.method(x_opt+shift, tol=1e-15)

## NEWTON-STAR

In [34]:
#run NEWTON-STAR
BM = Basic_method(oracle=logreg)
fv_bm, it_bm, bi_bm = BM.method(x_opt+shift, tol=1e-15)

## NEWTON-LEARN 1

In [46]:
# run NL1 for random-k with probability p
k = 1
p = 1/2
PCMP = PositiveCase_methodP(logreg)
x = x_opt+shift
H = []
for i in range(n):
    H.append(logreg.alphas(x,i))
fv_pcm, it_pcm, bi_pcm = PCMP.method(x, H, p=p, k=k, tol=1e-15)

## NEWTON-LEARN 2

In [48]:
# run NL2 for random-k with probability p
k = 1
p = 1/2

GCMP = GeneralCase_methodP(logreg)
x = x_opt+shift
H = []
for i in range(n):
    H.append(logreg.alphas(x,i))
fv_gcm, it_gcm, bi_gcm = GCMP.method(x, H, k=k, p=p,\
                                     gamma=1/2,\
                                     max_iter=10000, tol=1e-15)

## CUBIC-NEWTON-LEARN

In [14]:
# run CNL for random-k with probability p
k = 1
p = 1/2

CNLP = CubicMaxNewtonP(logreg)
x = x_opt+shift
H = []
for i in range(n):
    H.append(logreg.alphas(x,i))
fv_gcm, it_gcm, bi_gcm = CNLP.method(x, H, k=k, p=p,\
                                     nu=1/8, gamma=1/2,\
                                     max_iter=10, tol=1e-15)

## DINGO

In [15]:
# run DINGO 
Dingo = DINGO(logreg)
x = x_opt + shift
fv_dingo, bi_dingo, cases = Dingo.method(x)